In [3]:
import random

# --- ハミング符号化・復号 ---
def hamming_encode(data):
    d = [int(b) for b in data]
    p1 = d[0] ^ d[1] ^ d[3]
    p2 = d[0] ^ d[2] ^ d[3]
    p3 = d[1] ^ d[2] ^ d[3]
    return [p1, p2, d[0], p3, d[1], d[2], d[3]]

def hamming_decode(code):
    s1 = code[0] ^ code[2] ^ code[4] ^ code[6]
    s2 = code[1] ^ code[2] ^ code[5] ^ code[6]
    s3 = code[3] ^ code[4] ^ code[5] ^ code[6]
    error_position = s1 + (s2 << 1) + (s3 << 2)
    corrected = code.copy()
    if error_position != 0:
        corrected[error_position - 1] ^= 1
    return corrected, error_position

# --- CRCエンコード・チェック ---
def crc_encode(data, generator="10011"):
    data = data + "0000"
    data = list(data)
    g = list(generator)
    for i in range(len(data) - len(g) + 1):
        if data[i] == '1':
            for j in range(len(g)):
                data[i + j] = str(int(data[i + j]) ^ int(g[j]))
    return data[-4:]

def crc_check(full_data, generator="10011"):
    data = list(full_data)
    g = list(generator)
    for i in range(len(data) - len(g) + 1):
        if data[i] == '1':
            for j in range(len(g)):
                data[i + j] = str(int(data[i + j]) ^ int(g[j]))
    return data[-4:] == ['0'] * 4

# --- ビット列と誤り位置のテキスト可視化 ---
def print_bits_with_marker(label, bits, error_pos=None):
    bit_str = ''.join(str(b) for b in bits)
    marker = ' ' * error_pos + '^' if error_pos is not None else ''
    print(f"{label:20}: {bit_str}")
    if error_pos is not None:
        print(f"{'':20}  {marker}")

# --- 表の出力 ---
def print_table(rows):
    col_widths = [max(len(str(row[i])) for row in rows) for i in range(len(rows[0]))]
    print("-" * (sum(col_widths) + len(col_widths)*3 + 1))
    for row in rows:
        print("| " + " | ".join(f"{str(val).ljust(col_widths[i])}" for i, val in enumerate(row)) + " |")
        print("-" * (sum(col_widths) + len(col_widths)*3 + 1))

# --- メイン処理 ---

# ハミング符号処理
data = "1011"
encoded = hamming_encode(data)
error_pos = random.randint(0, 6)
received = encoded.copy()
received[error_pos] ^= 1
decoded, corrected_pos = hamming_decode(received.copy())

print("\n=== Hamming Code ===")
print_bits_with_marker("Encoded", encoded)
print_bits_with_marker("Received", received, error_pos)
print_bits_with_marker("Corrected", decoded, corrected_pos - 1 if corrected_pos else None)

# CRC処理
crc_data = "10111001"
crc_val = crc_encode(crc_data)
full_crc_data = list(crc_data) + crc_val

# ランダムに誤りを入れる
crc_error_data = full_crc_data.copy()
crc_err_pos = random.randint(0, len(full_crc_data) - 1)
crc_error_data[crc_err_pos] = '1' if crc_error_data[crc_err_pos] == '0' else '0'
crc_ok = crc_check(full_crc_data)
crc_ng = crc_check(crc_error_data)

print("\n=== CRC Code ===")
print_table([
    ["Type", "Bit Sequence", "Note"],
    ["Original Data", crc_data, ""],
    ["CRC Value", "".join(crc_val), "Appended to data"],
    ["Full Data", "".join(full_crc_data), "Sent to receiver"],
    ["Check (No Error)", "OK" if crc_ok else "Error", ""],
    ["Bit Flipped", "".join(crc_error_data), f"Error at bit {crc_err_pos + 1}"],
    ["Check (With Error)", "OK" if crc_ng else "Error", ""]
])



=== Hamming Code ===
Encoded             : 0110011
Received            : 0110010
                            ^
Corrected           : 0110011
                            ^

=== CRC Code ===
--------------------------------------------------------
| Type               | Bit Sequence | Note             |
--------------------------------------------------------
| Original Data      | 10111001     |                  |
--------------------------------------------------------
| CRC Value          | 1001         | Appended to data |
--------------------------------------------------------
| Full Data          | 101110011001 | Sent to receiver |
--------------------------------------------------------
| Check (No Error)   | OK           |                  |
--------------------------------------------------------
| Bit Flipped        | 101110011000 | Error at bit 12  |
--------------------------------------------------------
| Check (With Error) | Error        |                  |
------------